In [1]:
import pandas as pd
import numpy as np
import xgboost
from sklearn.base import TransformerMixin
from sklearn.tree import DecisionTreeClassifier

from sklearn import model_selection
from sklearn.metrics import accuracy_score
from swat import *
from swat.render import render_html
from pprint import pprint
from matplotlib import pyplot as plt
import pandas as pd
import sys
%matplotlib inline

cashost='localhost'
casport=5570
casauth='~/.authinfo'
sess = CAS(cashost, casport, authinfo=casauth, caslib="public")



In [2]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [22]:
df = pd.read_csv("/home/sasdemo01/data/HMEQ_MONITOR_ALL.csv",na_values=['NULL'])
targetvar = ['BAD']
inputvars = ['LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']
Y = df[targetvar]
X0 = df[inputvars]
# impute the missing value
X = DataFrameImputer().fit_transform(X0)

### Import the Python XGBOOST Model

In [23]:
import pickle
loaded_model = pickle.load(open("/home/sasdemo01/PythonModelXGBOOST/train/xgboost_model_obj.pkl", "rb"))

### Score New Data in Python

In [24]:
y_pred=loaded_model.predict_proba(X)
temp1=pd.DataFrame(y_pred,columns=["P_BAD0","P_BAD1"])
frames=[df,temp1]
HMEQ_MONITOR_P_ALL=pd.concat(frames,axis=1,ignore_index=False)
HMEQ_MONITOR_P_ALL.head(10)



,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,Period,P_BAD0,P_BAD1
0,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.894784,0.105216
1,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN,1,0.896627,0.103373
2,1,1700,30548.0,40320.0,HomeImp,Other,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614,1,0.273901,0.726099
3,1,1800,48649.0,57037.0,HomeImp,Other,5.0,3.0,2.0,77.100000,1.0,17.0,NaN,1,0.318281,0.681719
4,1,2000,22608.0,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0.729499,0.270501
5,1,2000,20627.0,29800.0,HomeImp,Office,11.0,0.0,1.0,122.533333,1.0,9.0,NaN,2,0.446295,0.553705
6,1,2000,45000.0,55000.0,HomeImp,Other,3.0,0.0,0.0,86.066667,2.0,25.0,NaN,1,0.858215,0.141785
7,0,2000,64536.0,87400.0,NaN,Mgr,2.5,0.0,0.0,147.133333,0.0,24.0,NaN,1,0.906593,0.093407
8,1,2200,23030.0,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,3.711312,4,0.493220,0.506780
9,1,2300,28192.0,40150.0,HomeImp,Other,4.5,0.0,0.0,54.600000,1.0,16.0,NaN,1,0.355348,0.644652


In [25]:
sess.upload_frame(HMEQ_MONITOR_P_ALL,casout={"caslib":"Public","promote":True,"name":"HMEQ_MONITOR_P_ALL"})

NOTE: Cloud Analytic Services made the uploaded file available as table HMEQ_MONITOR_P_ALL in caslib Public.
NOTE: The table HMEQ_MONITOR_P_ALL has been created in caslib Public from binary data uploaded to Cloud Analytic Services.


CASTable('HMEQ_MONITOR_P_ALL', caslib='Public')